<a href="https://colab.research.google.com/github/kihongmin/DL/blob/master/toxic_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://arxiv.org/pdf/1805.12307.pdf 적용 예정.
2020.04.21 -> preprocessing까지

In [0]:
import numpy as np
import pandas as pd

In [0]:
from tqdm import tqdm_notebook

In [0]:
train = pd.read_csv('train.csv.zip',compression='zip')
test = pd.read_csv('test.csv.zip', compression='zip')
test_label = pd.read_csv('test_labels.csv.zip',compression='zip')

In [0]:
y = train.iloc[:,2:].values

In [0]:
train.iloc[:,2:].sum(axis=0)

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [0]:
from collections import Counter

stop_eng = stopwords.words('english')

class makeData:
    def __init__(self, maxlen, embed_dim, minCount, mode='embedLayer'):
        self.maxlen = maxlen
        self.embed_dim = embed_dim
        self.minCount = minCount
        if mode not in ['embedLayer','FastText']:
            raise ValueError("Can not understand %s mode"%(mode))
        self.mode = mode
        self.preprocessed_train_title = 'preprocessed_train_'+str(maxlen)+'_'+str(embed_dim)+'.pkl'
        self.tokenized_train_title = 'tokenized_train_'+str(maxlen)+'_'+str(embed_dim)+'.pkl'
        self.preprocessed_test_title = 'preprocessed_test_'+str(maxlen)+'_'+str(embed_dim)+'.pkl'
        self.tokenized_test_title = 'tokenized_test_'+str(maxlen)+'_'+str(embed_dim)+'.pkl'
        self.word_list_name = 'word_list_'+mode+'.pkl'
        
    def fit(self,train):
        #전처리 끝난 데이터 있나요?
        if self.preprocessed_train_title in os.listdir():
            with open(self.preprocessed_train_title, 'rb') as f:
                preprocessed_train =  pickle.load(f)
            with open(self.word_list_name, 'rb') as f:
                self.word_list =  pickle.load(f)
            return preprocessed_train
        else:
            #토큰화 끝난 데이터 있나요?
            if self.tokenized_train_title in os.listdir():
                with open(self.tokenized_train_title, 'rb') as f:
                    train =  pickle.load(f)
            else:
                train = train['comment_text'].apply(lambda x:x.lower())
                print('apply word_tokenize')
                train = train.apply(word_tokenize)
                with open(self.tokenized_train_title, 'wb') as f:  
                    pickle.dump(train, f)
            #토큰화 종료 -> 전처리 시작
            print('start to preprocess raw data')
            train = self.preprocess(train,True)
            with open(self.preprocessed_train_title, 'wb') as f: 
                pickle.dump(train, f) 
            return train
        
    def predict(self, test):
        #전처리 끝난 데이터 있나요?
        if self.preprocessed_test_title in os.listdir():
            with open(self.preprocessed_test_title, 'rb') as f:
                preprocessed_test =  pickle.load(f)
            return preprocessed_test
        else:
            #토큰화 끝난 데이터 있나요?
            if self.tokenized_test_title in os.listdir():
                with open(self.tokenized_test_title, 'rb') as f:
                    test = pickle.load(f)
            else:
                test = test['comment_text'].apply(lambda x:x.lower())
                print('apply word_tokenize')
                test = test.apply(word_tokenize)
                with open(self.tokenized_test_title, 'wb') as f:  
                    pickle.dump(train, f)
            #토큰화 종료 -> 전처리 시작
            print('start to preprocess raw data')
            test = self.preprocess(test,False)
            with open(self.preprocessed_test_title, 'wb') as f: 
                pickle.dump(test, f) 
            return test
            
        
    def preprocess(self, df, is_train):
        p = re.compile('[A-Za-z0-9]+')
        df = df.apply(lambda x: list(filter(lambda y: y not in stop_eng, x)))
        df = df.apply(lambda x: list(filter(lambda y: p.match(y),x)))
        #train의 경우 word_list 생성 필요
        if is_train :            
        #모델에 임베딩 레이어 사용할 때
            print('start to make valid word list')
            if self.mode == 'embedLayer':
                counter = []
                for i in df:
                    counter.extend(i)
                counter = Counter(counter)
                self.word_list = dict(list(filter(lambda x:x[1]>5, counter.items())))
                with open(self.word_list_name, 'wb') as f:  
                    pickle.dump(self.word_list, f)
            #FastText로 임베딩된 워드 사용할 때
            elif self.mode == 'FastText':
                name = 'fasttext_'+str(self.maxlen)+'_'+str(self.embed_dim)+'.vec'
                if name in os.listdir():
                    self.ft_model = FastText.load(name)
                else:
                    self.ft_model = FastText(train, size=embed_dim, window=5, min_count=self.minCount, workers=4, sg=1)
                    self.ft_model.save(name)
                self.word_list = self.ft_model.wv.vocab.keys()
                with open(self.word_list_name, 'wb') as f:  
                    pickle.dump(self.word_list, f)
        #----------
        #word_list에 없는 단어들 버림
        print('start to filter words')

        for i, sentence in enumerate(df):
            df[i] = list(filter(lambda x: x in self.word_list, sentence))
        #이제는 mincount 이상의 빈도로 출현한 단어만 적용됨
        #-----------
        #문장의 단어 개수 통일
        
        print('start to fit shape')
        if self.mode == 'embedLayer':            
            tokenizer = Tokenizer()
            tokenizer.fit_on_texts(df)
            df = tokenizer.texts_to_sequences(df)
            df = pad_sequences(df, maxlen=self.maxlen)
            return df
                    
        elif self.mode == 'FastText':
            outlayer = np.zeros((len(df),self.maxlen,self.embed_dim))
            for i, sentence in enumerate(tqdm_notebook(df)):
                temp = np.zeros((self.maxlen,self.embed_dim))
                index = self.maxlen-len(sentence) if len(sentence) < self.maxlen else 0
                for word in sentence:
                    if index == self.maxlen:
                        break
                    temp[index] = self.ft_model.wv[word]
                    index+=1
                outlayer[i] = temp
            return outlayer        